As Peter does in his Grid-predict notebook, specify initial parameters from which to work.

In [ ]:
%matplotlib inline

import sys
sys.path.append('../..')

from matplotlib import pylab as plt

import itertools
import numpy
import scipy
import scipy.special
import time
import random

from ipywidgets import interact
from IPython.display import display, Markdown, clear_output

from crocodile.synthesis import *
from crocodile.simulate import *
from crocodile.antialias import *
from util.visualize import *
from arl.test_support import create_named_configuration, export_visibility_to_hdf5
from arl.data_models import *

In [ ]:
theta = 0.1
lam = 18000
grid_size = int(theta*lam)

In [ ]:
aa_over = 4096
aa_support = 6
aa_x0 = 0.499
aa_mode = 0
aa_szetan = True
if aa_support == 1:
    print("Using trivial gridder")
    aa_gcf = numpy.ones((aa_over, aa_support))
    def aa(x): return numpy.ones_like(x)
elif aa_szetan:
    print("Using Sze-Tan's gridder with R=%d, x_0=%g" % (aa_support//2, aa_x0))
    aa_gcf = sze_tan_gridder(aa_support//2, aa_x0, aa_over)
    def aa(x):
        return sze_tan_grid_correction_gen(aa_support//2, aa_x0, x)
    print("Mean error:", sze_tan_mean_error(aa_support//2, aa_x0))
else:
    aa_parameter = aa_support
    print("Using PSWF with mode %d and parameter %g" % (aa_mode, aa_parameter))
    aa_gcf = kernel_oversample(anti_aliasing_function(grid_size, aa_mode, aa_parameter), aa_over, aa_support)
    aa_gcf /= numpy.sum(aa_gcf[0])
    def aa(x):
        return scipy.special.pro_ang1(aa_mode, aa_mode, aa_parameter, 2*x)[0]
    
# Calculate appropriate step length to give us full accuracy for a field of view of size theta
du = du_opt = aa_x0/(theta/2)
print("Optimal du =", du)

# Plot gridding function
plt.rcParams['figure.figsize'] = 10, 5
r = numpy.arange(-aa_over*(aa_support//2), aa_over*((aa_support+1)//2)) / aa_over
plt.plot(du_opt*r, numpy.transpose(aa_gcf).flatten().real);
plt.xticks(du_opt*numpy.arange(-(aa_support//2), ((aa_support+1)//2)+1))
plt.grid(True);plt.xlabel('u/v [$\lambda$]');plt.title('$u/v$ Gridder');plt.show()

# Plot grid correction function
theta_x0 = theta/aa_x0/2
x = coordinates(10001)
#print(x)
sze_aa_im = aa(x)
print(sze_aa_im.shape)
plt.plot(theta*x/aa_x0/2, aa(x));
plt.title('$u/v$ Grid correction');plt.grid(True);plt.xlabel('l [1]')
plt.axvspan(theta/2, theta_x0/2, color='lightgray', hatch='x', alpha=0.5)
plt.axvspan(-theta/2, -theta_x0/2, color='lightgray', hatch='x', alpha=0.5)
plt.annotate('(unused)', xy=((theta+theta_x0)/4,0.9), ha='center', color='gray')
plt.annotate('(unused)', xy=(-(theta+theta_x0)/4,0.9), ha='center', color='gray');

In [ ]:
aa_support_w = 4
aa_x0_w = 0.125
aa_parameter_w = aa_support_w
aa_szetan_w = True
if aa_support_w == 1:
    print("Using trivial gridder")
    aa_gcf_w = numpy.ones((aa_over, aa_support_w))
    def aa_w(x): return numpy.ones_like(x)
elif aa_szetan_w:
    print("Using Sze-Tan's gridder with R=%d, x_0=%g" % (aa_support_w//2, aa_x0_w))
    aa_gcf_w = sze_tan_gridder(aa_support_w//2, aa_x0_w, aa_over)
    def aa_w(x):
        return sze_tan_grid_correction_gen(aa_support_w//2, aa_x0_w, x)
    print("Mean error:", sze_tan_mean_error(aa_support_w//2, aa_x0_w))
else:
    aa_gcf_w = kernel_oversample(anti_aliasing_function(grid_size, 0, aa_parameter_w), aa_over, aa_support_w)
    aa_gcf_w /= numpy.sum(aa_gcf_w[0])
    def aa_w(x):
        return scipy.special.pro_ang1(aa_mode, aa_mode, aa_parameter_w, 2*x)[0]

# Calculate appropriate step length to give us full accuracy for a field of view of size theta
max_n = 1.0 - numpy.sqrt(1.0 - 2*(theta/2)**2)
print("max_n =", max_n)
dw = dw_opt = aa_x0_w / max_n
print("Optimal dw =", dw)

# Plot gridding function
plt.rcParams['figure.figsize'] = 10, 5
r = numpy.arange(-aa_over*(aa_support_w//2), aa_over*((aa_support_w+1)//2)) / aa_over
plt.plot(dw_opt*r, numpy.transpose(aa_gcf_w).flatten().real);
plt.xticks(dw_opt*numpy.arange(-(aa_support_w//2), ((aa_support_w+1)//2)+1))
plt.grid(True); plt.xlabel('w [$\lambda$]'); plt.title('$w$ Gridder'); plt.show()

x = coordinates(10001)
print(x)
sze_w_im = aa_w(x)
print(sze_w_im)
plt.plot(max_n*x/aa_x0_w, aa_w(x));
plt.title('$w$ Grid correction'); plt.grid(True); plt.xlabel('$n$ [1]');
max_n_x0 = max_n/aa_x0_w/2
plt.axvspan(max_n, max_n_x0, color='lightgray', hatch='x', alpha=0.5)
plt.axvspan(-max_n, -max_n_x0, color='lightgray', hatch='x', alpha=0.5)
plt.annotate('(unused)', xy=((max_n+max_n_x0)/2,0.9), ha='center', color='gray')
plt.annotate('(unused)', xy=(-(max_n+max_n_x0)/2,0.9), ha='center', color='gray');

In [ ]:
Npt = 1000
points = theta * (numpy.random.rand(Npt,2)-0.5)
plt.rcParams['figure.figsize'] = 8, 8
plt.scatter(points[:,0], points[:,1]);

These are exact copies of peters functions in grid-predict. Used to compare with sze-tan w-stacking.

In [ ]:
def predict(dist_uvw, du=du_opt, dw=dw_opt, apply_aa = False, apply_aa_w = False):
    # Get image coordinates
    ls, ms = numpy.transpose(points)
    ns = numpy.sqrt(1.0 - ls**2 - ms**2) - 1
    # Evaluate grid correction functions in uv & w
    aas = numpy.ones(len(ls))
    if apply_aa:
         aas *= aa(du*ls) * aa(du*ms)
    if apply_aa_w:
        aas *= aa_w(dw*ns)
        # Now simulate points, dividing out grid correction
    vis = 0
    for l,m, a in zip(ls, ms, aas):
        vis += simulate_point(dist_uvw, l, m) / a
    return vis

def predict_grid(u,v,w,ov_u,ov_v,ov_w,du=du_opt, dw=dw_opt, visualise=False):
    
    # Generate offsets that we are going to sample at
    ius, ivs, iws = numpy.meshgrid(numpy.arange(aa_support), numpy.arange(aa_support), numpy.arange(aa_support_w))
    dus = du*(ius.flatten()-(aa_support//2)+ov_u/aa_over)
    dvs = du*(ivs.flatten()-(aa_support//2)+ov_v/aa_over)
    dws = dw*(iws.flatten()-(aa_support_w//2)+ov_w/aa_over)
    
    # Get grid convolution function for offsets
    aas = aa_gcf[ov_u,ius.flatten()] * aa_gcf[ov_v,ivs.flatten()] * aa_gcf_w[ov_w,iws.flatten()]

    # Add offsets to all uvw coordinates
    us = numpy.array(u)[:,numpy.newaxis] + dus[numpy.newaxis,:]
    vs = numpy.array(v)[:,numpy.newaxis] + dvs[numpy.newaxis,:]
    ws = numpy.array(w)[:,numpy.newaxis] + dws[numpy.newaxis,:]
    # Visualise sampling pattern?
    if visualise:
        ax = plt.subplot(111, projection='3d')
        ax.scatter(us,vs,ws, color='red');
        
        ax.set_xlabel('u'); ax.set_ylabel('v'); ax.set_zlabel('w')

    # Predict visibilities
    vis = predict(numpy.transpose([us.flatten(),vs.flatten(),ws.flatten()]),
                  du=du, dw=dw, apply_aa=True, apply_aa_w=True).reshape(us.shape)
    
    # Convolve with gridder, sum up
    #print(vis * aas[numpy.newaxis,:])
    #print(aas)
    return numpy.sum(vis * aas[numpy.newaxis,:], axis=1)

In [ ]:
def test(u=0,v=0,w=0, ov_u=0,ov_v=0,ov_w=0, du=du_opt, dw=dw_opt,visualise=True):
    print()
    vis = predict(numpy.transpose([[u],[v],[w]]))
    print("Direct: ", vis[0])
    vis_sum = predict_grid([u],[v],[w],ov_u,ov_v,ov_w,du,dw,visualise=visualise)
    print("Grid:   ", vis_sum[0])
    print("Error:  ", numpy.abs(vis[0]-vis_sum[0]) / numpy.sqrt(Npt))
print(du_opt)
print(dw_opt)    
test(u=1.0,v=1.0,w=1.0, ov_u=0,ov_v=0,ov_w=0, du=du_opt, dw=dw_opt,visualise=True)

In [ ]:
def generate_sky(theta, lam, points):
    grid_size = int(theta*lam)
    sky = numpy.zeros((grid_size,grid_size))
    points_coord = (((points + theta/2)/theta) * grid_size).astype(int)
    for point in points_coord:
        sky[point[0],point[1]] += 1.0
    return(sky)

def predict_dft(theta,lam,sky,u=0.0,v=0.0,w=0.0):
    grid_size = int(theta*lam)
    vis = 0.0
    for lp in numpy.arange(grid_size):
        l = float(lp)/float(grid_size) * theta - theta/2
        for mp in numpy.arange(grid_size):
            
            
            m = float(mp)/float(grid_size) * theta - theta/2
            n = numpy.sqrt(1 - l*l - m*m) - 1.0
            vis += sky[lp,mp] * numpy.exp (2j*numpy.pi*(u*l + v*m + w*n))
            

    return(vis)



def predict_wstack(theta,lam,points,x0,du=du_opt, dw=dw_opt,
                   gcf_support=aa_gcf, gcf_support_w=aa_gcf_w, aa=aa, aa_w=aa_w,
                   u=0.0, v=0.0, w=0.0, ov_u=0, ov_v=0, ov_w=0):
    grid_size = int(theta * lam)
    grid = numpy.zeros((grid_size, grid_size))
    ls, ms = numpy.meshgrid(numpy.arange(-theta/2,theta/2,theta/grid_size),numpy.arange(-theta/2,theta/2,theta/grid_size))
    ns = numpy.sqrt(1 - ls * ls - ms * ms) - 1.0

    #Fresnel Pattern
    ph = dw * (1 - numpy.sqrt(1 - ls * ls - ms * ms))
    fresnel_pattern = numpy.exp(1j*2*numpy.pi*ph)
    fig = plt.figure()
    fig.add_subplot(211)
    plt.imshow(fresnel_pattern.real)
    plt.colorbar()
    plt.title("Fresnel Pattern (Re)")
    fig.add_subplot(212)
    plt.imshow(fresnel_pattern.imag)
    plt.colorbar()
    plt.title("Fresnel Pattern (Im)")
    plt.tight_layout()
    plt.show()
    max_w = numpy.sin(theta/2) * numpy.sqrt(grid_size**2 + grid_size**2)
    print("Max W: ")
    print(max_w)
    w_planes = 2*numpy.ceil(max_w/(2*dw)) + 1 
    print(w_planes)
    sky = generate_sky(theta,lam, points)

    print("Computing DFT: ")
    vis = predict_dft(theta,lam,sky,u=u,v=v,w=w)
    print(vis)
    
    

    # Apply Grid Correction Function
    sky *= aa(du*ls)*aa(du*ms)*aa_w(dw*ns) # Grid Correction
    
    stacks = numpy.zeros(shape=(int(w_planes),grid_size,grid_size),dtype=numpy.complex64)

    # Transform Image to Bottom of Stack
    #for i in numpy.arange(numpy.floor(w_planes/2)+1):
    #    sky = sky * -fresnel_pattern
#    sky = sky * (fresnel_pattern ** -2)
    for i in numpy.arange(int(w_planes)):
        stacks[i,:,:] = numpy.fft.fftshift(numpy.fft.fft2(sky))
        sky = sky * fresnel_pattern
    
    # Now do reverse convolution using Sze Tan Kernels
   
    current_wp = numpy.ceil(w/dw)

    # Let's steal Peter's code because I know it works.
    ius, ivs, iws = numpy.meshgrid(numpy.arange(aa_support), numpy.arange(aa_support), numpy.arange(aa_support_w))
    dus = du*(ius.flatten()-(aa_support//2)+ov_u/aa_over)
    dvs = du*(ivs.flatten()-(aa_support//2)+ov_v/aa_over)
    dws = dw*(iws.flatten()-(aa_support_w//2)+ov_w/aa_over)
    
    # Get grid convolution function for offsets
    aas = aa_gcf[ov_u,ius.flatten()] * aa_gcf[ov_v,ivs.flatten()] * aa_gcf_w[ov_w,iws.flatten()]

    # Add offsets to all uvw coordinates
    us = numpy.array([u])[:,numpy.newaxis] + dus[numpy.newaxis,:]
    vs = numpy.array([v])[:,numpy.newaxis] + dvs[numpy.newaxis,:]
    ws = numpy.array([w])[:,numpy.newaxis] + dws[numpy.newaxis,:]

    wsp = (numpy.floor(ws/dw) + aa_support_w/2).astype(numpy.int)
    usp = (numpy.floor(us/du) + grid_size/2).astype(numpy.int)
    vsp = (numpy.floor(vs/du) + grid_size/2).astype(numpy.int)
    print(w_max)
    print(w_min)
    print(wsp)
    print(usp)
    print(aas.shape)
    vis = 0.0 + 1j*0.0
    for up,vp,wp,ap in zip(usp[0],vsp[0],wsp[0],aas):
        vis += stacks[wp,up,vp] * ap

    print(vis)

    
predict_wstack(theta,lam,points,aa_x0,u=0.0,v=0.0,w=0.0)

    
                                  
                                  
    